### Ejercicio 4
#### Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 
- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 
- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) Get familiar with data
2) Prepare the data for processing

3) Make action plan
4) Develop the code that works with a sample

5) Adjust the code to work with Big Data
6) Test big data approach on a sample

7) Run program with big data

#### hay muchas lineas duplicadas
Resulta que  hay lineas duplicadas en ambos archivos searches y bookings así que lo primero que hay que hacer es eliminarlos por chunks, y guardar un nuevo archivo de searches y bookings sin duplicados.

In [2]:
import pandas as pd
pd.options.display.max_columns=None

In [3]:
!ls

 10-11_Python_AmadChall_v19.pptx
 Amadeus_Challenge_Class.ipynb
 Amadeus_Challenge_Test.ipynb
 HowToTackleDataScienceChallenge.ipynb
 bookings.3columns.sorted.bz2
 bookings.3columns.sorted2.bz2
 bookings.csv.bz2
 bookings.sample.csv.bz2
 ch_01-Empty.ipynb
 ch_02-Empty.ipynb
 ch_03-MINE.ipynb
 ch_04-MINE.ipynb
 ch_05-MINE.ipynb
 ch_05-MINE_RemoteConectionToWebServer.ipynb
 ch_05-empty.ipynb
 searches.csv.bz2
 top_airports.csv
'~$10-11_Python_AmadChall_v19.pptx'


In [6]:
%%time
bi=pd.read_csv('bookings.csv.bz2', sep="^", chunksize=2000000, low_memory=False, dtype=str)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)

all_chunks.to_csv('bookings_wn_dup.csv', sep='^', index=False)

0 0
1 1000000
2 1000000
3 1000003
4 1000003
5 1000003
CPU times: user 4min 1s, sys: 23.4 s, total: 4min 24s
Wall time: 4min 3s


In [7]:
%%time
bi=pd.read_csv('searches.csv.bz2', sep="^", chunksize=2000000, low_memory=False, dtype=str)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)

all_chunks.to_csv('searches_wn_dup.csv', sep='^', index=False)

0 0
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
CPU times: user 5min 9s, sys: 32.8 s, total: 5min 42s
Wall time: 5min 2s


Bookings columns: 'act_date           ', 'dep_port', 'arr_port'
Searches columns: 'Date', 'Origin', 'Destination'

#### we have seen in the previous exercise that there are white spaces in the column names

In [9]:
#get 3 columns of interest de bookings file
b=pd.read_csv('bookings_wn_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'])

In [11]:
#get the whole searches file
s=pd.read_csv('searches_wn_dup.csv', sep='^', low_memory=False)

In [12]:
b.columns

Index(['act_date           ', 'dep_port', 'arr_port'], dtype='object')

In [13]:
b.columns=b.columns.str.strip()

In [14]:
b.columns

Index(['act_date', 'dep_port', 'arr_port'], dtype='object')

In [15]:
#ahora quitamos white spaces de los valores de las columnas
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()

In [16]:
b.arr_port[0]

'LHR'

#### Cómo es nuestro modelo?
Vamos a simplificar al máximo.
Eliminaremos duplicados de los bookings, miraremos a searches fila por fila, si existe una reserva que coincide con el search entonces cumple con nuestro modelo.

#### Action plan
1) quitar duplicados de bookings
2) preparar act_date column de bookings para que el formato coincida con Date de searches
3) añado a bookings una columna con '1' unos 
#así resuelvo el problema de las reservas. Las búsquedas sin reservas quedarán NaN.
4) merge
5) fillna(0) over the result. Rellenamos con '0' ceros los NaN


In [17]:
b.shape

(1000003, 3)

In [18]:
b=b.drop_duplicates()
b.shape

(466228, 3)

In [19]:
b['Booked?']=1
b.head()

/home/angelrps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,act_date,dep_port,arr_port,Booked?
0,2013-03-05 00:00:00,ZRH,LHR,1
1,2013-03-26 00:00:00,SAL,CLT,1
3,2013-03-26 00:00:00,AKL,SVO,1
5,2013-03-20 00:00:00,DEN,LGA,1
7,2013-03-25 00:00:00,NRT,SIN,1


In [28]:
b['act_date']=b['act_date'].str[:10]

/home/angelrps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
b.head()

,act_date,dep_port,arr_port,Booked?
0,2013-03-05,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
5,2013-03-20,DEN,LGA,1
7,2013-03-25,NRT,SIN,1


In [35]:
s_b= s.merge(b,
        how='left',
        left_on=['Date', 'Origin', 'Destination'],
        right_on=['act_date', 'dep_port', 'arr_port'])

In [37]:
#sanity check
#si hemos hecho bien el merge las líneas de s deberían ser las mismas que s_b
print(s.shape, s_b.shape)

(359004, 45) (359004, 49)


In [38]:
#las columnas ['act_date', 'dep_port', 'arr_port'] nos sobran así que las eliminamos
s_b=s_b.drop(['act_date', 'dep_port', 'arr_port'], axis=1)
s_b

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Booked?
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,NaN
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,NaN
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,NaN
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358999,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN,NaN
359000,"2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359001,2013-01-01,18:04:49,MPT,3561,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,NaN
359002,2013-01-01,19:57:57,MPT,28d7a8c95e4db88589d3d35b66920e78,DE,FRA,BGW,1.0,2.0,FRA,BGW,2013-02-26,NaN,NaN,BGW,FRA,2013-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BNJ,NaN,NaN,NaN


In [41]:
s_b['Booked?']=s_b['Booked?'].fillna(0)

In [42]:
s_b.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Booked?
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


In [44]:
s_b[s_b['Booked?']==1].shape
#salen 17524 filas con '1' unos. Estos son los searches que terminaron en booking

(17514, 46)